In [ ]:
# MySQL是Web世界中使用最广泛的数据库服务器。
# SQLite的特点是轻量级、可嵌入，但不能承受高并发访问，适合桌面和移动应用。
# 而MySQL是为服务器端设计的数据库，能承受高并发访问，同时占用的内存也远远大于SQLite。

# 此外，MySQL内部有多种数据库引擎，最常用的引擎是支持数据库事务的InnoDB。

In [1]:
# 安装MySQL
# 可以直接从MySQL官方网站下载最新的Community Server 8.x版本。MySQL是跨平台的，选择对应的平台下载安装文件，安装即可。
# 安装时，MySQL会提示输入root用户的口令，请务必记清楚。如果怕记不住，就把口令设置为password。

# 在Windows上，安装时请选择UTF-8编码，以便正确地处理中文。
# 在Mac或Linux上，需要编辑MySQL的配置文件，把数据库默认的编码全部改为UTF-8。
# MySQL的配置文件默认存放在/etc/my.cnf或者/etc/mysql/my.cnf：

In [2]:
'''
[client]
default-character-set = utf8mb4

[mysqld]
default-storage-engine = INNODB
character-set-server = utf8mb4
collation-server = utf8_general_ci
'''

'\n[client]\ndefault-character-set = utf8mb4\n\n[mysqld]\ndefault-storage-engine = INNODB\ncharacter-set-server = utf8mb4\ncollation-server = utf8_general_ci\n'

In [3]:
# 重启MySQL后，可以通过MySQL的客户端命令行检查编码：
'''
$ mysql -u root -p
Enter password: 
Welcome to the MySQL monitor...
...

mysql> show variables like '%char%';
+--------------------------+--------------------------------------+
| Variable_name            | Value                                |
+--------------------------+--------------------------------------+
| character_set_client     | utf8mb4                              |
| character_set_connection | utf8mb4                              |
| character_set_database   | utf8mb4                              |
| character_set_filesystem | binary                               |
| character_set_results    | utf8mb4                              |
| character_set_server     | utf8mb4                              |
| character_set_system     | utf8mb3                              |
| character_sets_dir       | /usr/local/mysql-8.x/share/charsets/ |
+--------------------------+--------------------------------------+
8 rows in set (0.00 sec)
'''

"\n$ mysql -u root -p\nEnter password: \nWelcome to the MySQL monitor...\n...\n\nmysql> show variables like '%char%';\n+--------------------------+--------------------------------------+\n| Variable_name            | Value                                |\n+--------------------------+--------------------------------------+\n| character_set_client     | utf8mb4                              |\n| character_set_connection | utf8mb4                              |\n| character_set_database   | utf8mb4                              |\n| character_set_filesystem | binary                               |\n| character_set_results    | utf8mb4                              |\n| character_set_server     | utf8mb4                              |\n| character_set_system     | utf8mb3                              |\n| character_sets_dir       | /usr/local/mysql-8.x/share/charsets/ |\n+--------------------------+--------------------------------------+\n8 rows in set (0.00 sec)\n"

In [5]:
# 看到utf8mb4字样就表示编码设置正确。
# 如果MySQL的版本<5.5.3，则只能把编码设置为utf8，
# utf8mb4支持最新的Unicode标准，可以显示emoji字符，但utf8无法显示emoji字符。

# 用Docker启动MySQL
# 如果不想安装MySQL，还可以以Docker的方式快速启动MySQL。
# 首先安装Docker Desktop，然后在命令行输入：
'''
$ docker run -e MYSQL_ROOT_PASSWORD=password -p 3306:3306 
                --name mysql-8.4 -v ./mysql-data:/var/lib/mysql mysql:8.4 
                --mysql-native-password=ON 
                --character-set-server=utf8mb4 -
                -collation-server=utf8mb4_unicode_ci
'''

'\n$ docker run -e MYSQL_ROOT_PASSWORD=password -p 3306:3306 \n                --name mysql-8.4 -v ./mysql-data:/var/lib/mysql mysql:8.4 \n                --mysql-native-password=ON \n                --character-set-server=utf8mb4 -\n                -collation-server=utf8mb4_unicode_ci\n'

In [6]:
# 上述命令详细参数如下：

# -e MYSQL_ROOT_PASSWORD=password：传入root用户口令的环境变量，密码是password；
# -p 3306:3306：在本机3306端口监听；
# --name mysql-8.4：启动后容器的名称为mysql-8.4，可任意设置；
# -v ./mysql-data:/var/lib/mysql：把当前目录./mysql-data映射到容器目录/var/lib/mysql，此目录存放MySQL数据库文件，避免容器停止后数据丢失；
# mysql:8.4：启动镜像名称为mysql:8.4；
# --mysql-native-password=ON：表示启用明文口令；
# --character-set-server=utf8mb4：表示启用utf8mb4作为字符集；
# --collation-server=utf8mb4_unicode_ci：表示启用utf8mb4作为排序规则。

# 看到最后一行ready for connections表示启动成功。

In [7]:
# 安装MySQL驱动
# 由于MySQL服务器以独立的进程运行，并通过网络对外服务，
# 所以，需要支持Python的MySQL驱动来连接到MySQL服务器。MySQL官方提供了mysql-connector-python驱动：

# $ pip install mysql-connector-python 

In [8]:
# 我们演示如何连接到MySQL服务器的test数据库：
'''
# 导入MySQL驱动:
>>> import mysql.connector
# 注意把password设为你的root口令:
>>> conn = mysql.connector.connect(user='root', password='password', database='test')
>>> cursor = conn.cursor()
# 创建user表:
>>> cursor.execute('create table user (id varchar(20) primary key, name varchar(20))')
# 插入一行记录，注意MySQL的占位符是%s:
>>> cursor.execute('insert into user (id, name) values (%s, %s)', ['1', 'Michael'])
>>> cursor.rowcount
1
# 提交事务:
>>> conn.commit()
>>> cursor.close()
# 运行查询:
>>> cursor = conn.cursor()
>>> cursor.execute('select * from user where id = %s', ('1',))
>>> values = cursor.fetchall()
>>> values
[('1', 'Michael')]
# 关闭Cursor和Connection:
>>> cursor.close()
True
>>> conn.close()
'''

"\n# 导入MySQL驱动:\n>>> import mysql.connector\n# 注意把password设为你的root口令:\n>>> conn = mysql.connector.connect(user='root', password='password', database='test')\n>>> cursor = conn.cursor()\n# 创建user表:\n>>> cursor.execute('create table user (id varchar(20) primary key, name varchar(20))')\n# 插入一行记录，注意MySQL的占位符是%s:\n>>> cursor.execute('insert into user (id, name) values (%s, %s)', ['1', 'Michael'])\n>>> cursor.rowcount\n1\n# 提交事务:\n>>> conn.commit()\n>>> cursor.close()\n# 运行查询:\n>>> cursor = conn.cursor()\n>>> cursor.execute('select * from user where id = %s', ('1',))\n>>> values = cursor.fetchall()\n>>> values\n[('1', 'Michael')]\n# 关闭Cursor和Connection:\n>>> cursor.close()\nTrue\n>>> conn.close()\n"

In [ ]:
# 由于Python的DB-API定义都是通用的，所以，操作MySQL的数据库代码和SQLite类似。

# 小结
# 执行INSERT等操作后要调用commit()提交事务；
# MySQL的SQL占位符是%s。